In [1]:
!pip install pycocotools
!pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp38-cp38-win_amd64.whl.metadata (34 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.6.2-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.6.2-py3-none-any.whl (164 kB)
Using cached charset_normalizer-3.3.2-cp38-cp38-win_amd64.whl (99 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)


In [22]:
from pycocotools.coco import COCO

annFile = 'annotations_trainval2017\\annotations\\instances_train2017.json'

coco = COCO(annFile)

loading annotations into memory...
Done (t=34.55s)
creating index...
index created!


In [94]:
data_size=50

union_classes = ['bicycle', 'kite', 'zebra', 'backpack', 'keyboard', 'laptop', 'microwave', 'mouse', 'bench', 'parking meter', 'refrigerator', 'toaster', 'umbrella', 'vase', 'traffic light', 'broccoli', 'orange', 'banana', 'pizza', 'cup']

catIds = coco.getCatIds(catNms=union_classes)
print(catIds)

for union_class in union_classes:
    catId = coco.getCatIds(catNms=union_class)
    print(f'{union_class}: {catId}')

imgIds = set()
for catId in catIds:
    imgId = coco.getImgIds(catIds=catId)
    for img in imgId:
        imgIds.add(img)
imgIds_dataset = imgIds
print(len(imgIds))

[2, 10, 14, 15, 24, 27, 28, 38, 47, 52, 55, 56, 59, 73, 74, 76, 78, 80, 82, 86]
bicycle: [2]
kite: [38]
zebra: [24]
backpack: [27]
keyboard: [76]
laptop: [73]
microwave: [78]
mouse: [74]
bench: [15]
parking meter: [14]
refrigerator: [82]
toaster: [80]
umbrella: [28]
vase: [86]
traffic light: [10]
broccoli: [56]
orange: [55]
banana: [52]
pizza: [59]
cup: [47]
45705


In [78]:
import os

if not os.path.exists('dataset'):
    os.makedirs('dataset/images')
    os.makedirs('dataset/labels')

In [93]:
import requests
count = 0

def get_ann_classes(img):
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)
    classes = set()
    for ann in anns:
            class_id = ann['category_id']
            classes.add(str(class_id))
    return classes

for imgId in imgIds_dataset:
    img = coco.loadImgs(imgId)[0]
    url = img['coco_url']
    img_data = requests.get(url).content
    classes = get_ann_classes(img)
    if len(classes) != 1 or '70' in classes:
        continue

    with open(f"dataset/images/{img['file_name']}", 'wb') as handler:
        handler.write(img_data)

    with open(f"dataset/labels/{img['file_name'][:-4]}.txt", 'w') as f:
        f.write(' '.join(classes))
    
    count += 1
    if count >= data_size:
        break

In [39]:
classes = set()

directory_path = 'dataset/labels'

files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

for file_name in files:
    file_path = os.path.join(directory_path, file_name)
    with open(file_path, 'r') as file:
        content = file.read().split(' ')
        for item in content:
            classes.add(int(item))

print(sorted(classes))

[2, 4, 5, 6, 10, 11, 15, 16, 19, 20, 25, 52, 81, 85, 86]


In [46]:
cats = coco.loadCats(coco.getCatIds())
coco_classes = {cat['id']: cat['name'] for cat in cats}

def get_class_name(class_id):
    return coco_classes.get(class_id, "Unknown")

class_names = [get_class_name(class_id) for class_id in classes]

print(class_names)
print(f'Count: {len(class_names)}')

['bicycle', 'motorcycle', 'airplane', 'bus', 'traffic light', 'fire hydrant', 'bench', 'bird', 'sink', 'horse', 'banana', 'sheep', 'vase', 'clock', 'giraffe']
Count: 15


In [64]:
import urllib

all_coco_classes = []

with open("coco_classes.txt", 'r') as file:
    content = file.read().split("\n")
    for item in content:
        all_coco_classes.append(item)
all_coco_classes = all_coco_classes[1:-1]

url = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'
response = urllib.request.urlopen(url)
imagenet_classes = [line.strip() for line in response]

map = []

for idx, class_name in enumerate(imagenet_classes):
    for item in all_coco_classes:
        if item in str(class_name):
            map.append(f'ImageNet: {idx}, {class_name}, COCO: {all_coco_classes.index(item)}, {item}')
            
with open(f"coco_imageNet_class_map.txt", 'w') as f:
    f.write("\n".join(map))

In [52]:
for idx, class_name in enumerate(imagenet_classes):
    for item in class_names:
        if item in str(class_name):
            print(f'Index: {idx}, Class: {class_name}')

Index: 94, Class: b'hummingbird'
Index: 138, Class: b'bustard'
Index: 229, Class: b'Old English sheepdog'
Index: 230, Class: b'Shetland sheepdog'
Index: 375, Class: b'colobus'
Index: 409, Class: b'analog clock'
Index: 444, Class: b'bicycle-built-for-two'
Index: 448, Class: b'birdhouse'
Index: 530, Class: b'digital clock'
Index: 603, Class: b'horse cart'
Index: 654, Class: b'minibus'
Index: 703, Class: b'park bench'
Index: 779, Class: b'school bus'
Index: 874, Class: b'trolleybus'
Index: 883, Class: b'vase'
Index: 892, Class: b'wall clock'
Index: 920, Class: b'traffic light'
Index: 954, Class: b'banana'


In [ ]:
imageNet_to_coco_map = {
    444: 2, # bicycle
    21: 38, # kite
    340: 24, # zebra
    414: 27, # backpack
    508: 76, # keyboard
    620: 73, # laptop 
    651: 78, # microwave
    673: 74, # mouse
    703: 15, # bench
    704: 14, # parking meter
    760: 82, # refrigerator
    859: 80, # toaster
    879: 28, # umbrella
    883: 86, # vase
    920: 10, # traffic light
    937: 56, # broccoli
    950: 55, # orange
    954: 52, # banana
    963: 59, # pizza
    968: 47, # cup
}